In [1]:
import os
os.environ["OPENAI_API_BASE"] = "https://api.chatanywhere.tech"
os.environ["OPENAI_API_KEY"] = 'sk-mvzdD655Ttsdn1C7IFKvdPN3XKtY5QI64ScWtiYmM4Le0K7i'

In [2]:
# 导入Document Loaders
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, TextLoader

# 加载Documents
base_dir = 'Docs' # 文档的存放目录
documents = []
for file in os.listdir(base_dir): 
    # 构建完整的文件路径
    file_path = os.path.join(base_dir, file)
    if file.endswith('.pdf'):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.docx'):
        loader = Docx2txtLoader(file_path)
        documents.extend(loader.load())
    elif file.endswith('.txt'):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# 将Documents切分成块以便后续进行嵌入和向量存储
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)

# 将分割嵌入并存储在矢量数据库Qdrant中
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
vectorstore = Qdrant.from_documents(
    documents=chunked_documents, # 以分块的文档
    embedding=OpenAIEmbeddings(), # 用OpenAI的Embedding Model做嵌入
    location=":memory:",  # in-memory 存储
    collection_name="my_documents" # 指定collection_name
)

In [3]:
# 准备模型和Retrieval链
import logging # 导入Logging工具
from langchain_openai import ChatOpenAI # ChatOpenAI模型
from langchain.retrievers.multi_query import MultiQueryRetriever # MultiQueryRetriever工具
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain


# 设置Logging
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)

# 实例化一个大模型工具 - OpenAI的GPT-3.5
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# 实例化一个MultiQueryRetriever
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(), llm=llm)

# 实例化一个memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()

# 实例化一个ConversationalRetrievalChain
qa_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever_from_llm, memory=memory, verbose=True)

/var/folders/z5/r_6tsnmx7bjcnqbgkmfp22540000gn/T/ipykernel_88182/3197553982.py:20: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [4]:
result = qa_chain({"question": "什么是MCP？"})
print(result)
print(result["answer"])

/var/folders/z5/r_6tsnmx7bjcnqbgkmfp22540000gn/T/ipykernel_88182/688091923.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"question": "什么是MCP？"})
INFO:langchain.retrievers.multi_query:Generated queries: ['MCP是什么？  ', 'MCP的定义和概念是什么？  ', 'MCP具体指的是什么内容？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
MCP调研报告

1. 什么是MCP（Model Context Protocol）

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）

提供哪些服务：

Tools：提供给LLM应用特别是Agent使用的工具。

Resoures：提供给LLM应用一些额外的结构化数据。

Prompts：提供给LLM应用的一些Prompt模板。比如你的应用是一个Chatbot，可以从MCP Server中取出这些模板，让使用者选择使用。

使用什么语言创建？

目前支持TS/Python/Java SDK来编写MCP Server。

这也就意味着，如果我们要发开一个 Function Call 工具，需要对不同的模型进行适配，比如参数格式、触发逻辑、返回结构等等，这个成本是非常高的。

阶段三

In [5]:
# 具有记忆功能，根据上下文推断“它”为MCP
print(qa_chain({"question": "它的使用方法是什么？"}))
print(result)
print(result["answer"])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 什么是MCP？
Assistant: MCP是Model Context Protocol（模型上下文协议）的简称，是由Anthropic在2024年11月底推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。其主要目的在于解决当前AI模型由于数据孤岛限制而无法充分发挥潜力的难题，使得AI应用能够安全地访问和操作本地及远程数据，为AI应用提供连接各种资源的接口。
Follow Up Input: 它的使用方法是什么？
Standalone question:

> Finished chain.


INFO:langchain.retrievers.multi_query:Generated queries: ['MCP的具体使用步骤有哪些？  ', '如何正确使用MCP？  ', 'MCP的操作指南是什么？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Command 对应命令，args 对应参数。对于公开的 MCP 工具，我们通常可以这样找到相应的设置方法，后面我还会介绍如何查找更多公开的 MCP 服务器。

4.2.4 使用方法

安装完服务器后，在对话界面点击启用 MCP 服务器。

选择支持 MCP 的模型（带有扳手 🔧 标志）。

就可以开始对话了！



使用mcp功能



选择支持mcp的模型

对话效果：



对话效果

MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）

MCP调研报告

1. 什么是MCP（Model Context Protocol）

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

工具操作文件夹，实现通过对话控制桌面文件。通过这些示例，旨在帮助用户了解 MCP 强大的扩展性。
Human: MCP的使用方法是什么？

> Finish

In [6]:
# 创建 Map-Reduce 摘要链
from langchain.chains.summarize import load_summarize_chain

# 使用 OpenAI 的 GPT-3.5 模型实例化摘要链，采用 map_reduce 策略
summ_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=True)

# 定义一个函数，接收用户问题，检索相关文档并生成摘要
def qa_with_summary(question: str):
    # 使用多查询检索器获取与问题相关的文档
    docs = retriever_from_llm.get_relevant_documents(question)
    # 对检索到的文档进行摘要
    summary = summ_chain.run(docs)
    return summary

# 示例：生成关于 MCP 核心概念的摘要
print(qa_with_summary("请用 100 字总结 MCP 的核心概念"))

/var/folders/z5/r_6tsnmx7bjcnqbgkmfp22540000gn/T/ipykernel_88182/2417023693.py:10: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever_from_llm.get_relevant_documents(question)
INFO:langchain.retrievers.multi_query:Generated queries: ['请简要概述 MCP 的主要思想，控制在 100 字以内。  ', '能否用 100 字解释一下 MCP 的基本概念？  ', '请用不超过 100 字的方式描述 MCP 的核心内容。']
/var/folders/z5/r_6tsnmx7bjcnqbgkmfp22540000gn/T/ipykernel_88182/2417023693.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = summ_chain.run(docs)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"工具操作文件夹，实现通过对话控制桌面文件。通过这些示例，旨在帮助用户了解 MCP 强大的扩展性。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP调研报告

1. 什么是MCP（Model Context Protocol）"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"提供哪些服务：

Tools：提供给LLM应用特别是Agent使用的工具。

Resoures：提供给LLM应用一些额外的结构化数据。

Prompts：提供给LLM应用的一些Prompt模板。比如你的应用是一个Chatbot，可以从MCP Server中取出这些模板，让使用者选择使用。

使用什么语言创建？

目前支持TS/Python/Java SDK来编写MCP Server。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"MCP 客户端（

/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"MCP (Model Context Protocol) is an open standard launched by Anthropic in late November 2024, designed to unify communication between large language models (LLMs) and external data sources and tools. Its primary goal is to address the limitations of data silos that hinder AI models' potential, enabling secure access and operation of local and remote data, thereby providing a comprehensive interface for AI applications.

The document demonstrates how to use a tool to manage desktop files through dialogue, showcasing the powerful extensibility of MCP.

The MCP (Model Context Protocol) is a framework designed to facilitate the interaction and communication between different models in a system, ensuring that they operate effectively within their respective contexts.

The service offers tools for LLM applications, structured data resources, and prompt templates for app

In [9]:
# 导入所需的工具类和Agent初始化方法
from langchain.tools import Tool
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent, AgentType

# 定义一个函数，用于通过Retrieval QA链回答用户的问题
def retriever_tool_func(q):
    return qa_chain({"question": q})["answer"]  # 调用qa_chain处理问题，并提取答案部分

# 创建一个用于文档检索问答的工具
retrieval_tool = Tool(name="RAG_QA", func=retriever_tool_func, description="对一般知识类问题，先检索文档再回答")

# 创建一个用于文档摘要的工具
summary_tool = Tool(name="RAG_Summary", func=qa_with_summary, description="当用户要求概括/总结时使用")

# 创建一个用于执行Python代码的工具
python_tool = PythonREPLTool()

# 将上述工具组合成一个工具列表，供Agent使用
tools = [retrieval_tool, summary_tool, python_tool]

# 初始化一个智能体（Agent），配置如下：
# - 使用OpenAI的GPT-3.5模型（llm）
# - 提供的工具列表（tools）
# - 使用ZERO_SHOT_REACT_DESCRIPTION类型的Agent（基于ReAct框架）
# - 启用详细日志输出（verbose=True）
# - 使用会话记忆（memory）以保持上下文
agent = initialize_agent(
    tools=tools,  # 提供的工具列表
    llm=llm,  # 使用的语言模型
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Agent的类型
    verbose=True,  # 是否输出详细的日志信息
    memory=memory  # 会话记忆，用于保持对话的上下文
)

/var/folders/z5/r_6tsnmx7bjcnqbgkmfp22540000gn/T/ipykernel_88182/431357988.py:28: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [10]:
print(agent.run("MCP 的作用能一步总结给我吗？"))    # ➜ 触发 RAG_Summary 工具



> Entering new AgentExecutor chain...
MCP 可能指代多种事物，但通常在技术和计算机科学中，它指的是“微控制器平台”（Microcontroller Platform）或“多通道处理”（Multi-Channel Processing）。为了准确总结其作用，我需要确认具体的上下文。由于问题没有提供具体的背景，我将进行文档检索以获取更准确的信息。

Action: RAG_QA
Action Input: "MCP 的作用"


> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 什么是MCP？
Assistant: MCP是Model Context Protocol（模型上下文协议）的简称，是由Anthropic在2024年11月底推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。其主要目的在于解决当前AI模型由于数据孤岛限制而无法充分发挥潜力的难题，使得AI应用能够安全地访问和操作本地及远程数据，为AI应用提供连接各种资源的接口。
Human: 它的使用方法是什么？
Assistant: MCP的使用方法如下：

1. 安装完服务器后，在对话界面点击启用MCP服务器。
2. 选择支持MCP的模型（带有扳手 🔧 标志）。
3. 就可以开始对话了！ 

通过这些步骤，您可以使用MCP功能与支持的模型进行交互。
Follow Up Input: MCP 的作用"

Standalone question:

> Finished chain.


INFO:langchain.retrievers.multi_query:Generated queries: ['MCP 的功能是什么？  ', 'MCP 在什么方面发挥作用？  ', 'MCP 的主要用途是什么？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
MCP调研报告

1. 什么是MCP（Model Context Protocol）

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

MCP 客户端（MCP Clients）：由客户端LLM应用使用Client SDK创建并维护的一个Server会话。

MCP 服务器（MCP Servers）：为Client  提供上下文、工具和 prompt 等信息的服务。可以部署在LLM应用本机，也可以远程部署（Remote）。



3.2 工作流程：

[该类型的内容暂不支持下载]

MCP 服务器（MCP Servers）

Command 对应命令，args 对应参数。对于公开的 MCP 工具，我们通常可以这样找到相应的设置方法，后面我还会介绍如何查找更多公开的 MCP 服务器。

4.2.4 使用方法

安装完服务器后，在对话界面点击启用 MCP 服务器。

选择支持 MCP 的模型（带有扳手 🔧 标志）。

就可以开始对话了！



使用mcp功能



选择支持mcp的模型

对话效果：



对话效果

这也就意味着，如果我们要发开一个 Function Call 工具，需要对不同的模型进行适配，比如参数格式、触发逻辑、返回结构等等，这个成本是非常高的。



In [11]:
print(agent.run("计算 (27*13)+9 等于多少？"))   # ➜ 触发 Python 解释器工具



> Entering new AgentExecutor chain...


我需要计算表达式 (27*13)+9 的值。  
Action: Python_REPL  
Action Input: (27 * 13) + 9  
Observation: 
Thought:我现在知道了最终答案。  
Final Answer: 360

> Finished chain.
360


In [12]:
print(agent.run("MCP 和 LangChain Output Parser 有何区别？"))   # ➜ 触发 RAG_QA 工具



> Entering new AgentExecutor chain...
我需要查找有关 MCP 和 LangChain Output Parser 的信息，以便了解它们之间的区别。  
Action: RAG_QA  
Action Input: "MCP 和 LangChain Output Parser 有何区别？"  

> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: 什么是MCP？
Assistant: MCP是Model Context Protocol（模型上下文协议）的简称，是由Anthropic在2024年11月底推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。其主要目的在于解决当前AI模型由于数据孤岛限制而无法充分发挥潜力的难题，使得AI应用能够安全地访问和操作本地及远程数据，为AI应用提供连接各种资源的接口。
Human: 它的使用方法是什么？
Assistant: MCP的使用方法如下：

1. 安装完服务器后，在对话界面点击启用MCP服务器。
2. 选择支持MCP的模型（带有扳手 🔧 标志）。
3. 就可以开始对话了！ 

通过这些步骤，您可以使用MCP功能与支持的模型进行交互。
Human: MCP 的作用"

Assistant: MCP（Model Context Protocol）的作用是统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。它旨在解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的问题，使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供连接万物的接口。具体来说，MCP 提供了工具、结构化数据和提示模板等服务，帮助 LLM 应用更好地进行交互和处理信息。
Human: MCP 的作用能一步总结给我吗？
Assistant: MCP（Model Con

INFO:langchain.retrievers.multi_query:Generated queries: ['MCP 和 LangChain 的输出解析器之间有什么不同之处？  ', 'MCP 与 LangChain 的输出解析器相比，主要的区别是什么？  ', '请问 MCP 和 LangChain 的输出解析器各自的特点和差异是什么？']




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Model返回的内容通常都是字符串的模式，但在实际开发过程中，往往希望model可以返回更直观的内容，Langchain提供的输出解析器则将派上用场。

在实现一个输出解析器的过程中，需要实现两种方法：

获取格式指令：返回一个字符串的方法，其中包含有关如何格式化语言模型输出的说明。

Parse：一种接收字符串（假设是来自语言模型的响应）并将其解析为某种结构的方法。

这也就意味着，如果我们要发开一个 Function Call 工具，需要对不同的模型进行适配，比如参数格式、触发逻辑、返回结构等等，这个成本是非常高的。

阶段三、MCP（ Anthropic  在 2024 .11 提出）















MCP

核心贡献：统一规范

新的AI能力共享生态：通过MCP Server的共享，新的LLM应用可以快速获得各种工具，形成了一种新的合作体系，提高整体效用。

总结：FunctionCall和MCP的区别



3. 核心架构

3.1 概念解析：



MCP 主机（MCP Hosts）：发起请求的 LLM 应用程序（例如 Claude Desktop、IDE 或 AI 工具）。

MCP（Model Context Protocol，模型上下文协议） ，2024年11月底，由 Anthropic 推出的一种开放标准，旨在统一大型语言模型（LLM）与外部数据源和工具之间的通信协议。MCP 的主要目的在于解决当前 AI 模型因数据孤岛限制而无法充分发挥潜力的难题，MCP 使得 AI 应用能够安全地访问和操作本地及远程数据，为 AI 应用提供了连接万物的接口。

MCP调研报

In [13]:
from langchain_core.callbacks import BaseCallbackHandler

class SimpleCallbackHandler(BaseCallbackHandler):
    """一个最小可用的回调示例：打印 LLM / Chain 的关键阶段"""

    # LLM 开始生成时触发
    def on_llm_start(self, serialized, prompts, **kwargs):
        print("\n🔵 LLM started")  # serialized 是模型信息；prompts 是提示词

    # LLM 流式生成新 token 时触发（若模型/链支持流式）
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)  # 实时打印每个 token，可做打字机效果

    # LLM 结束时触发
    def on_llm_end(self, response, **kwargs):
        print("\n🟢 LLM finished")  # response 包含完整文本及 token 使用量

    # 整个链（Chain / Agent）结束时触发
    def on_chain_end(self, outputs, **kwargs):
        # outputs 是一个 dict，键值由具体链决定
        print(f"🟡 Chain outputs: {list(outputs.keys())}")

In [15]:
callback_handler = SimpleCallbackHandler()

# callbacks 可以同时传入多个 handler 的列表
agent.run("LangChain 的作用能一步总结给我吗？", callbacks=[callback_handler])



> Entering new AgentExecutor chain...

🔵 LLM started



🟢 LLM finished
🟡 Chain outputs: ['text']
LangChain 是一个用于构建基于语言模型的应用程序的框架。它提供了一系列工具和组件，帮助开发者更容易地集成和使用语言模型，支持多种功能，如文本生成、问答、对话系统等。为了更好地总结，我将使用 RAG_Summary 工具来获取更详细的信息。

Action: RAG_Summary
Action Input: "LangChain 的作用"
🔵 LLM started


INFO:langchain.retrievers.multi_query:Generated queries: ['LangChain 的功能是什么？  ', 'LangChain 可以做些什么？  ', 'LangChain 的主要用途有哪些？']



🟢 LLM finished


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（一）：入门

介绍

LangChain 是一个用于开发由语言模型驱动的应用程序的框架。它使得应用程序能够：

具有上下文感知能力：将语言模型连接到上下文来源（提示指令，少量的示例，需要回应的内容等）

具有推理能力：依赖语言模型进行推理（根据提供的上下文如何回答，采取什么行动等）

这个框架由几个部分组成。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 库：Python 和 JavaScript 库。包含了各种组件的接口和集成，一个基本的运行时，用于将这些组件组合成链和代理，以及现成的链和代理的实现。

LangChain 模板：一系列易于部署的参考架构，用于各种任务。

LangServe：一个用于将 LangChain 链部署为 REST API 的库。"


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangChain 系列教程（二）：Model I/O

介绍

LangChain 提供的 LangChain Expression Language(LCEL) 让开发可以很方便地将多个组件连接成 AI 工作流。如下是一个简单的工作流："


CONCISE SUMMARY:
Prompt after formatting:
Write a concise summary of the following:


"LangSmith：一个开发者平台，让你可以调试、测试、评估和监控基于任何 LLM 框架构建的链，并且与 LangChain 无缝集成。 

核心模块

LangChain由6个module组成，分别是

/opt/anaconda3/envs/zjh/lib/python3.10/site-packages/langchain_openai/chat_models/base.py:386: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟢 LLM finished

🟢 LLM finished
🟡 Chain outputs: ['outputs']

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"LangChain is a framework designed for developing applications powered by language models. It enables applications to have context awareness by connecting language models to various context sources and to perform reasoning based on the provided context. The framework consists of several components.

LangChain is a library for Python and JavaScript that provides interfaces and integrations for various components, a runtime for combining them into chains and agents, and ready-made implementations. It includes templates for easily deployable reference architectures for different tasks and LangServe, a library for deploying LangChain chains as REST APIs.

The second tutorial in the LangChain series, titled "Model I/O," introduces the LangChain Expression Languag

'LangChain 是一个用于开发基于语言模型的应用程序的框架，提供了多种组件和工具，支持上下文感知和推理。它包括六个核心模块：模型输入输出、检索、链、记忆、代理和回调，帮助开发者集成和部署 AI 工作流，并通过 LangServe 提供 REST API。'